In [ ]:
import warnings
import collections
#please save these files in the same directory inside folder ./Data
Indir='./'
Org_File='Data_file.csv'
X_Data_Name='x_data.csv'
Y_Data_Name='y_data.csv'
Cats='cat_cols.csv'
Cons='con_cols.csv'
savedir='./Figures/Missing_Variables'

In [ ]:
MisPerc=['0','10','20','30','40','50','60','70','80','90']
imputers=['MICE']
Models=['xgboost','adaboost','randomforest','linearreg']
ComRes=collections.defaultdict()
j=1
end=10
import numpy as np
warnings.simplefilter('ignore',DeprecationWarning)
for model in Models:
    for imputer in imputers:
        for missing_percentage in MisPerc:
            Results=collections.defaultdict()
            print '#'*40+' Run '+str(j)+' '+'#'*40
            from ReadData import Read_Data as DataRead
            Data_Read=DataRead()
            #-------------Loading Data-----------------
            Data=Data_Read.load_data(infile=Org_File,snxdata=X_Data_Name,snydata=Y_Data_Name,sncatcols=Cats,snconcols=Cons,indir=Indir)
            #-------------Filter Missing Data----------
            Proc_Data=Data_Read.filter_missing_data(x_data=Data['x_data'],missing_percentage=missing_percentage,cat_cols=Data['cat_cols'],con_cols=Data['con_cols'])
            #-------------Normalize and categorize Data----------
            train_data=Data_Read.normalize_categorize_only(x_filtered_w_missing=Proc_Data['x_data'],new_con_cols=Proc_Data['con_cols'],new_cat_cols=Proc_Data['cat_cols'])

            from TrainData import Train_Data as TrainData
            #-------------Impute Data and Train Model----------
            Data_Train=TrainData()
            Output=Data_Train.train_model(x_data=train_data,y_data=Data['y_data'],cat_cols=Proc_Data['cat_cols'],con_cols=Proc_Data['con_cols'],model=model,imputer=imputer,cvsplit=4,rstate=101,misper=missing_percentage)
            ModelNum='Model_'+str(j)
            Results['MisPerc']=missing_percentage
            Results['ValAcc']=Output['Mean_vals']['Val']['Acc']
            Results['TestAcc']=Output['Mean_vals']['Test']['Acc']
            Results['model']=model
            Results['TPR']=Output['Acc_vals']['Val']['TPR']
            Results['FPR']=Output['Acc_vals']['Val']['FPR']
            Results['imputer']=imputer
            print 'Completed '+str(model)+' with missing '+str(missing_percentage)+' imputer '+str(imputer)
            ComRes[ModelNum]=Results        
            j=j+1
savename='Results_'+str(model)+str(3)+'.npy'
np.save(savename,np.array(ComRes))
